In [21]:
from pandas import read_csv
import tensorflow as tf
import numpy as np
from elasticsearch import Elasticsearch
import pickle 
#import numpy as np


In [4]:
df = read_csv('./features.csv')
df.head()

,ID,Label,Sequence,hydrophobicity_polar_first,hydrophobicity_polar_20%,hydrophobicity_polar_40%,hydrophobicity_polar_60%,hydrophobicity_polar_80%,hydrophobicity_polar_100%,hydrophobicity_neutral_first,...,solvent_accessibility_intermediate_40%,solvent_accessibility_intermediate_60%,solvent_accessibility_intermediate_80%,solvent_accessibility_intermediate_100%,solvent_accessibility_exposed_first,solvent_accessibility_exposed_20%,solvent_accessibility_exposed_40%,solvent_accessibility_exposed_60%,solvent_accessibility_exposed_80%,solvent_accessibility_exposed_100%
0,WP_395757889.1,AMP,MPYDSVYLEKRPPGALRTVWRKFYGDTTAMIGLYGCAGLLLLCVFG...,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000,0.006757,...,0.371622,0.540541,0.736486,0.932432,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000
1,WP_395755974.1,AMP,MKILIVEDDTLIQQGLAQAMARENYAYDCADSAVGASALLQSSHYS...,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909,0.045455,...,0.377273,0.645455,0.845455,1.000000,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909
2,WP_395732316.1,AMP,MDFSRKPTQDPEAWLDFAPEFSQSMARQVRGWIQRWEPDLAESIKW...,0.009852,0.182266,0.418719,0.605911,0.862069,0.995074,0.019704,...,0.399015,0.630542,0.783251,0.950739,0.009852,0.182266,0.418719,0.605911,0.862069,0.995074
3,WP_395723680.1,AMP,MDCSRTPTRDPETWLDLAPEFSQAMARQVRAWIQRWEPDLTESIKW...,0.009852,0.192118,0.477833,0.655172,0.866995,0.995074,0.019704,...,0.325123,0.497537,0.689655,0.921182,0.009852,0.192118,0.477833,0.655172,0.866995,0.995074
4,WP_395695240.1,AMP,MADRDTAGVTFETTVSSFGNNTGIEVPEEVIERLGRGKRPPVLVTV...,0.019231,0.205128,0.455128,0.717949,0.891026,1.000000,0.012821,...,0.346154,0.532051,0.762821,0.884615,0.019231,0.205128,0.455128,0.717949,0.891026,1.000000


In [9]:
df['Vector'] = list(df[df.columns[3:]].values)

In [11]:
df.head(2)

,ID,Label,Sequence,hydrophobicity_polar_first,hydrophobicity_polar_20%,hydrophobicity_polar_40%,hydrophobicity_polar_60%,hydrophobicity_polar_80%,hydrophobicity_polar_100%,hydrophobicity_neutral_first,...,solvent_accessibility_intermediate_60%,solvent_accessibility_intermediate_80%,solvent_accessibility_intermediate_100%,solvent_accessibility_exposed_first,solvent_accessibility_exposed_20%,solvent_accessibility_exposed_40%,solvent_accessibility_exposed_60%,solvent_accessibility_exposed_80%,solvent_accessibility_exposed_100%,Vector
0,WP_395757889.1,AMP,MPYDSVYLEKRPPGALRTVWRKFYGDTTAMIGLYGCAGLLLLCVFG...,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000,0.006757,...,0.540541,0.736486,0.932432,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000,"[0.0135135135135135, 0.1891891891891892, 0.304..."
1,WP_395755974.1,AMP,MKILIVEDDTLIQQGLAQAMARENYAYDCADSAVGASALLQSSHYS...,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909,0.045455,...,0.645455,0.845455,1.000000,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909,"[0.009090909090909, 0.2545454545454545, 0.4409..."


In [18]:

client = Elasticsearch('http://localhost:9200')
client.ping()

True

In [6]:
mapping = {
    'properties': {
        'Id': {
            'type': 'text'
        },
        'Label': {
            'type': 'text',
        },
        'Sequence': {
            'type': 'text',
        },
        'Vector': {
            'type': 'dense_vector',
            'dims': 126,
            'similarity': 'l2_norm'
        }
    }
}
for col in df.columns[3:]:
    mapping['properties'][col] = {'type': 'double'}
mapping

{'properties': {'Id': {'type': 'text'},
  'Label': {'type': 'text'},
  'Sequence': {'type': 'text'},
  'Vector': {'type': 'dense_vector', 'dims': 126, 'similarity': 'l2_norm'},
  'hydrophobicity_polar_first': {'type': 'double'},
  'hydrophobicity_polar_20%': {'type': 'double'},
  'hydrophobicity_polar_40%': {'type': 'double'},
  'hydrophobicity_polar_60%': {'type': 'double'},
  'hydrophobicity_polar_80%': {'type': 'double'},
  'hydrophobicity_polar_100%': {'type': 'double'},
  'hydrophobicity_neutral_first': {'type': 'double'},
  'hydrophobicity_neutral_20%': {'type': 'double'},
  'hydrophobicity_neutral_40%': {'type': 'double'},
  'hydrophobicity_neutral_60%': {'type': 'double'},
  'hydrophobicity_neutral_80%': {'type': 'double'},
  'hydrophobicity_neutral_100%': {'type': 'double'},
  'hydrophobicity_hydrophobic_first': {'type': 'double'},
  'hydrophobicity_hydrophobic_20%': {'type': 'double'},
  'hydrophobicity_hydrophobic_40%': {'type': 'double'},
  'hydrophobicity_hydrophobic_60%':

In [13]:
if client.indices.exists(index= 'genes'):
    client.indices.delete(index='genes')
client.indices.create(index = 'genes', mappings= mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'genes'})

In [15]:
df.head()

,ID,Label,Sequence,hydrophobicity_polar_first,hydrophobicity_polar_20%,hydrophobicity_polar_40%,hydrophobicity_polar_60%,hydrophobicity_polar_80%,hydrophobicity_polar_100%,hydrophobicity_neutral_first,...,solvent_accessibility_intermediate_60%,solvent_accessibility_intermediate_80%,solvent_accessibility_intermediate_100%,solvent_accessibility_exposed_first,solvent_accessibility_exposed_20%,solvent_accessibility_exposed_40%,solvent_accessibility_exposed_60%,solvent_accessibility_exposed_80%,solvent_accessibility_exposed_100%,Vector
0,WP_395757889.1,AMP,MPYDSVYLEKRPPGALRTVWRKFYGDTTAMIGLYGCAGLLLLCVFG...,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000,0.006757,...,0.540541,0.736486,0.932432,0.013514,0.189189,0.304054,0.631757,0.804054,1.000000,"[0.0135135135135135, 0.1891891891891892, 0.304..."
1,WP_395755974.1,AMP,MKILIVEDDTLIQQGLAQAMARENYAYDCADSAVGASALLQSSHYS...,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909,0.045455,...,0.645455,0.845455,1.000000,0.009091,0.254545,0.440909,0.636364,0.831818,0.990909,"[0.009090909090909, 0.2545454545454545, 0.4409..."
2,WP_395732316.1,AMP,MDFSRKPTQDPEAWLDFAPEFSQSMARQVRGWIQRWEPDLAESIKW...,0.009852,0.182266,0.418719,0.605911,0.862069,0.995074,0.019704,...,0.630542,0.783251,0.950739,0.009852,0.182266,0.418719,0.605911,0.862069,0.995074,"[0.0098522167487684, 0.1822660098522167, 0.418..."
3,WP_395723680.1,AMP,MDCSRTPTRDPETWLDLAPEFSQAMARQVRAWIQRWEPDLTESIKW...,0.009852,0.192118,0.477833,0.655172,0.866995,0.995074,0.019704,...,0.497537,0.689655,0.921182,0.009852,0.192118,0.477833,0.655172,0.866995,0.995074,"[0.0098522167487684, 0.1921182266009852, 0.477..."
4,WP_395695240.1,AMP,MADRDTAGVTFETTVSSFGNNTGIEVPEEVIERLGRGKRPPVLVTV...,0.019231,0.205128,0.455128,0.717949,0.891026,1.000000,0.012821,...,0.532051,0.762821,0.884615,0.019231,0.205128,0.455128,0.717949,0.891026,1.000000,"[0.0192307692307692, 0.2051282051282051, 0.455..."


In [16]:
bulk = []
for row in df.to_dict('records'):
    bulk.append({'index': {'_id': row['ID']}})
    bulk.append(row)

In [19]:
client.bulk(index='genes', body=bulk)

ObjectApiResponse({'errors': False, 'took': 735510689, 'items': [{'index': {'_index': 'genes', '_id': 'WP_395757889.1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6000, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'genes', '_id': 'WP_395755974.1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6001, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'genes', '_id': 'WP_395732316.1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6002, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'genes', '_id': 'WP_395723680.1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6003, '_primary_term': 1, 'status': 200}}, {'index': {'_index': 'genes', '_id': 'WP_395695240.1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'fa

In [20]:
client.count(index='genes')

ObjectApiResponse({'count': 6000, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [43]:
#from sklearn.ensemble import RandomForestClassifier

with open('model.pkl', 'rb') as file:
    # Call load method to deserialze 
    model = pickle.load(file) 

print(model)

RandomForestClassifier(max_depth=30, n_estimators=9000, random_state=22)


In [23]:
import math
properties = {
    "hydrophobicity": {
        "polar": "RKEDQN",
        "neutral": "GASTPHY",
        "hydrophobic": "CLVIMFW"
    },
    "vdw_volume": {
        "small": "GASTPDC",
        "medium": "NVEQIL",
        "large": "MHKFRYW"
    },
    "polarity": {
        "polar": "EDQNKR",
        "neutral": "GASTPHY",
        "nonpolar": "CLVIMFW"
    },
    "polarizability": {
        "low": "GASDT",
        "medium": "CPNVEQIL",
        "high": "KMHFRYW"
    },
    "charge": {
        "negative": "DE",
        "neutral": "ACFGHILMNPQSTVWY",
        "positive": "KR"
    },
    "secondary_structure": {
        "helix": "EALMQKRH",
        "sheet": "VIYCWT",
        "coil": "GNPSD"
    },
    "solvent_accessibility": {
        "buried": "ALFCGIVW",
        "intermediate": "MPSTHY",
        "exposed": "DEKNQR"
    }
}

def calculate_distribution(sequence, amino_class):
    length = len(sequence)
    class_residues = [i for i, aa in enumerate(sequence) if aa in amino_class]
    total = len(class_residues)
    dic = { "first": 0, "20%": 0.2, "40%": 0.4, "60%": 0.6, "80%": 0.8, "100%": 1.0 }
    distribution = {}
    for k in dic:
        pos = min(math.floor(total * dic[k]), total - 1)
        redidule_pos = class_residues[pos]
        distribution[k] = (redidule_pos + 1) / length
    return distribution

def to_features(sequence):
    feature_dict = {}
    for prop, classes in properties.items():
            for class_name, amino_class in classes.items():
                # Calculate the distribution descriptors
                distribution = calculate_distribution(sequence, amino_class)
                for key, value in distribution.items():
                    feature_name = f"{prop}_{class_name}_{key}"
                    feature_dict[feature_name] = value
    return feature_dict

In [42]:
seq = 'MRNRLNRLGLESKSHYKLYKSGRRWVAASITVFSVGIGLTFSQVEQVKAATGTGVDTADNSASVSSDMAEPSNAVVLKSASTATATKTATQDAKAATDVTAATQDTKATTDSTGATSASSNRQSTAATKPAAEVGTASSSADSSASISSTDGASASAPSVTSKSTNTEATSASATKTATTSADTDVLNTETTSSSVANDLTDATTASQTRTETGKTASIPTAEAPTITTAVTSRALPLTGALASRSANTPVTKSAVQAVSAITSEAETKPTVSLVTTGTVSMDYGEASLADLESHISSPDETPANDVAYYIQDAAGNYLEDVNGNKVNLLYALFLDSADVNDYVDVVYTDEHGQVTKYSGDTDFSTLDQIGSYSVTINAAGKAGMSRVMQDYNAYDTSTSDLDDFVPTFSTGASDYTFTINIVPVKITATTGKNGLIILRPSQLYTGSLTMLPVVTVKNATKQNILQISNGEIGDAKPGVAGKVGQRVLTLADFTYTYQGTETNLTGADTGKYAITLNDAGRKAVQAALGSNYILDDAAVFTTTGAVQAAGLELKIASGTVTYNGKPQGTTVAITAGTAYDHFDFTTTTDTNVGTYDDLTYALADPTQAAILAKNYTVTTTDGTLVITPADLTVTVKDDNPVYDGRAHGTTATVTSGTNYDQLAFTAVAADGSGATTYTTVGTYAMTGTTAADTSNYKISYVNGTLTIDPAKATITIPNKIYWSDGTQKNLAAVVTGTVNGETLKYRVTNGMSAVGTKTIMATPDADDSVNKNYTISVIPGTLTIGDIAVKYLYEHVDANGETQVDASETGTATHATDATATDYLTYTTAAKPKTGYALAPNTGLAYNGTLTDQGGTVTYRYLAKTETAIVTYFDQTDNKVIKTDPLQGTYGTTDAYRTADTIAAYENAGYDLVSDDYPTAGVVYDQDDIVQKYQVTLVHKFVTRTPDNPGTPGEPIDPDNPNGPTYPVGTDF'
dic = to_features(seq)
vec = list(dic.values())
vec

[0.0020554984583761563,
 0.20760534429599178,
 0.40184994861253853,
 0.6063720452209661,
 0.8221993833504625,
 0.998972250770812,
 0.009249743062692703,
 0.17985611510791366,
 0.38335046248715315,
 0.6032887975334018,
 0.7985611510791367,
 0.9979445015416238,
 0.0010277492291880781,
 0.2692702980472765,
 0.434737923946557,
 0.5899280575539568,
 0.7821171634121274,
 1.0,
 0.009249743062692703,
 0.1778006166495375,
 0.3751284686536485,
 0.5950668036998972,
 0.789311408016444,
 0.998972250770812,
 0.003083247687564234,
 0.25488180883864336,
 0.43268242548818087,
 0.579650565262076,
 0.7995889003083247,
 0.9958890030832477,
 0.0010277492291880781,
 0.26002055498458376,
 0.4450154162384378,
 0.6546762589928058,
 0.8376156217882836,
 1.0,
 0.0020554984583761563,
 0.20760534429599178,
 0.40184994861253853,
 0.6063720452209661,
 0.8221993833504625,
 0.998972250770812,
 0.009249743062692703,
 0.17985611510791366,
 0.38335046248715315,
 0.6032887975334018,
 0.7985611510791367,
 0.997944501541623

In [48]:
result = client.search(index='genes', body ={
    "knn": {
        "field": "Vector",
        "query_vector": vec,
        "k": 3,
        "num_candidates": 100,
        'filter': {
            'term': {
                'Label': 'AMP'
            }
        }
    }
}, filter_path='hits.hits')
for row in result['hits']['hits']:
    print(row)

In [44]:
d = ['nAMP', 'AMP']
pred = model.predict([vec])
d[pred[0]]

c:\ProgramData\anaconda3\envs\tktmdt\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'nAMP'